# <font size="7"><b>Bayesian Methods of Hyperparameter Optimization</b></font>

In addition to the $\small\textbf{random search}$ and the $\small\textbf{grid search}$ methods for selecting optimal hyperparameters, we can use $\small\textbf{Bayesian methods}$ of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

# How Bayesian optimization works?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

---

# 1. Setup

## 1.1. Modules

> <b>NOTE:</b> You will need to install the <b><code>bayesian-optimization</code></b> , <b><code>catboost</code></b> , and <b><code>lightgbm</code></b> modules. Running a cell with an exclamation point in the beginning of the command will run it as a shell command.

In [134]:
! pip install bayesian-optimization
! pip install catboost
# ! pip install lightgbm

## 1.2. Dependencies

In [179]:
import os
from os.path import join

import warnings
import lightgbm
import numpy as np
import pandas as pd
from functools import partial
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, cv, Pool

In [136]:
warnings.filterwarnings('ignore')

In [137]:
os.listdir()

['.config', 'drive', 'sample_data']

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.3. Functions

### <font size="6"><b><code>label_encode_categorical_columns()</code></b></font>

In [139]:
def label_encode_categorical_columns(df):

    """
    Encode labels of categorical columns in a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing categorical columns.

    Returns:
    - pd.DataFrame: DataFrame with categorical columns label-encoded.
    """

    # Identify columns with dtype 'object' as potential categorical columns
    categorical_cols = df\
                      .select_dtypes(include=['object'])\
                      .columns\
                      .tolist()

    # Encode labels for each categorical column
    for col in categorical_cols:
        if col in df.columns:
            df[col] = LabelEncoder().fit_transform(df[col])

    return df, categorical_cols

### <font size="6"><b><code>make_harmonic_features()</code></b></font>

In [140]:
def make_harmonic_features(
      x
    , type
    , period = 2400
):

    """
    Calculate the sine or cosine of the angle of x on a circle with a given period.

    Parameters:
    - x (float): Value to calculate the sine or cosine for.
    - type (str): Type of trigonometric function to calculate ('sin' or 'cos').
    - period (int): Period of the circle.

    Returns:
    - float: The sine or cosine of the angle depending on the "type" argument.
    """
    angle = 2.0 * np.pi * x / period

    if type == "sin":
        return np.sin(angle)
    elif type == "cos":
        return np.cos(angle)
    else:
        raise ValueError("Invalid 'type' argument. Use 'sin' or 'cos'.")

### <font size="6"><b><code>feature_eng()</code></b></font>

In [141]:
def feature_eng(df):

    """
    Perform feature engineering on a given DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with flight-related data.

    Returns:
    - pd.DataFrame: DataFrame with engineered features.
    """

    # Create a new feature 'flight' by combining 'Origin' and 'Dest'
    df['flight'] = df['Origin'] + df['Dest']

    # Extract month from 'Month' and convert to int
    df['Month'] = df['Month']\
                  .map(lambda x: int(x.split('-')[-1]))\
                  .astype('int32')

    # Extract day of the month from 'DayofMonth' and categorize it
    df['DayofMonth']       =   df['DayofMonth'].map(lambda x: int(x.split('-')[-1])).astype('uint8')
    df['begin_of_month']   = ( df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10) & (df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month']     = ( df['DayofMonth'] >= 20).astype('uint8')

    # Extract day of the week from 'DayOfWeek' and categorize it
    df['DayOfWeek'] = df['DayOfWeek']\
                      .map(lambda x: int(x.split('-')[-1]))\
                      .astype('uint8')

    # Extract hour from 'DepTime' and categorize it
    df['hour']    = (df['DepTime'] / 100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11) & (x >= 7)  else 0).astype('uint8')
    df['day']     = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night']   = df['hour'].map(lambda x: 1 if (x >= 0)  & (x <= 6)  else 0).astype('int32')

    # Categorize seasons based on 'Month'
    df['winter'] = df['Month'].map(lambda x: int(x in [12, 1, 2] )).astype('int32')
    df['spring'] = df['Month'].map(lambda x: int(x in [3, 4, 5]  )).astype('int32')
    df['summer'] = df['Month'].map(lambda x: int(x in [6, 7, 8]  )).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: int(x in [9, 10, 11])).astype('int32')

    # Create binary features for 'holiday' and 'weekday'
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int)
    df['weekday'] = (df['DayOfWeek'] < 5 ).astype(int)

    # Create features based on counts of airports and carriers
    df['airport_dest_per_month']   = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count']       = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count']     = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count']            = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per_month']  = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')

    # Create harmonic features for 'DepTime'
    df['deptime_cos'] = df['DepTime'].map(lambda x: make_harmonic_features(x, type="cos"))
    df['deptime_sin'] = df['DepTime'].map(lambda x: make_harmonic_features(x, type="sin"))

    # Create features by combining 'flight' and 'UniqueCarrier'
    df['flightUC'] = df['flight'] + df['UniqueCarrier']
    df['DestUC']   = df['Dest']   + df['UniqueCarrier']
    df['OriginUC'] = df['Origin'] + df['UniqueCarrier']

    # Drop 'DepTime' column as it is no longer needed
    return df.drop('DepTime', axis=1)

### <font size="6"><b><code>lgb_eval()</code></b></font>

In [142]:
def lgb_eval(
      num_leaves
    , max_depth
    , lambda_l2
    , lambda_l1
    , min_child_samples
    , min_data_in_leaf
    , train_data
    , train_label
    , categorical_features
):

    """
    Perform LightGBM model evaluation with various hyperparameters.

    Parameters:
    - num_leaves (int): The maximum number of leaves in each tree.
    - max_depth (int): The maximum depth of the trees in the LightGBM model.
    - lambda_l2 (float): L2 regularization term to prevent overfitting.
    - lambda_l1 (float): L1 regularization term to prevent overfitting.
    - min_child_samples (int): Minimum number of data points required to form a leaf in a tree.
    - min_data_in_leaf (int): Alias for min_child_samples. Minimum number of data points required to form a leaf in a tree.
    - X_train (array-like): Training data features.
    - y_train (array-like): Training data labels.
    - categorical_features (list): List of categorical feature indices.

    Returns:
    - AUC Score (float): The final AUC (Area Under the ROC Curve) score obtained after cross-validation.
    """

    params = {
          "objective"         : "binary"
        , "metric"            : "auc"
        , "is_unbalance"      : True
        , "num_leaves"        : int(num_leaves)
        , "max_depth"         : int(max_depth)
        , "lambda_l2"         : lambda_l2
        , "lambda_l1"         : lambda_l1
        , "num_threads"       : 20
        , "min_child_samples" : int(min_child_samples)
        , "min_data_in_leaf"  : int(min_data_in_leaf)
        , "learning_rate"     : 0.03
        , "subsample_freq"    : 5
        , "bagging_seed"      : 42
        , "verbosity"         : -1
    }

    lgtrain = lightgbm.Dataset(
          train_data
        , train_label
        , categorical_feature = cat_cols
    )

    cv_result = lightgbm.cv(
          params
        , lgtrain
        , num_boost_round = 1000
        , stratified      = True
        , nfold           = 3
    )

    return cv_result['valid auc-mean'][-1]

---

# 2. Preprocessing

### Load

You can load the <b>zipped csv</b> or <b><code>.csv.zip</code></b> files just as you would regular <b><code>.csv</code></b> files using Pandas <b><code>read_csv</code></b> . In the next cell load the train and test data into two seperate dataframes.


In [143]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/data/'

train_df = pd.read_csv(join(dir_path, '18.2.6.flight_delays_train.csv.zip'))
test_df  = pd.read_csv(join(dir_path, '18.2.6.flight_delays_test.csv.zip'))

Print the top five rows (head) of the train dataframe <b><code>train_df</code></b> and review the columns in the data.

In [144]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


### Overview

In [145]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


### Summary Statitsics

In [146]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


> <b>NOTE:</b> The `DepTime` feature is the departure time in a numeric representation in <b><code>2400</code></b> hours.

### Target

The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.

In [147]:
# train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min']\
          .map({'Y': 1, 'N': 0})\
          .values

### Feature Engineering

Concatenate the training and testing dataframes.


In [148]:
full_df = pd.concat(
    [
          train_df.drop('dep_delayed_15min', axis=1)
        , test_df
    ]
)

Use the function <b><code>feature_eng()</code></b> to create additional features for the model. To do this, apply the function to the full dataframe.

In [149]:
full_df = feature_eng(
    full_df
)

In [150]:
full_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per_month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,AA,ATL,DFW,732,ATLDFW,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,ATLDFWAA,DFWAA,ATLAA
1,4,20,3,US,PIT,MCO,834,PITMCO,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,PITMCOUS,MCOUS,PITUS
2,9,2,5,XE,RDU,CLE,416,RDUCLE,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,RDUCLEXE,CLEXE,RDUXE
3,11,25,6,OO,DEN,MEM,872,DENMEM,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,DENMEMOO,MEMOO,DENOO
4,10,7,6,WN,MDW,OMA,423,MDWOMA,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,MDWOMAWN,OMAWN,MDWWN


### Label Encoding

Use the functions <b><code>label_encode_categorical_columns()</code></b> to encode the labels within each column. The function returns the full encoded dataframe as well as the list of names of all categorical columns.

In [151]:
full_df_encoded, cat_cols = label_encode_categorical_columns(full_df)

Check how the dataset looks like after encoding of labels within each column. First, extract the list of names of all the categorical features. We will need this list when performing the cross-validation on the LightGBM model using <b><code>lightgbm.cv()</code></b> inside the ustom function <b><code>lgb_eval()</code></b> we defined earlier.

In [152]:
print(f"List of names of categorical columns: {cat_cols}")

List of names of categorical columns: ['UniqueCarrier', 'Origin', 'Dest', 'flight', 'flightUC', 'DestUC', 'OriginUC']


In [153]:
full_df_encoded.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per_month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103



Split the new full encoded dataframe <b><code>full_df_encoded</code></b> into <b><code>X_train</code></b> and <b><code>X_test</code></b> datasets.

In [154]:
X_train = full_df_encoded[:train_df.shape[0]]
X_test  = full_df_encoded[train_df.shape[0]:]

---

# 3. Models (+ BayesOpt)

## 3.1. Simple Model

The first step is to create an instance from the <b><code>BayesianOptimization</code></b> optimizer. It uses two items:
* <b>Target Function</b> to optimize
* <b>Hyperparameters Search Space</b> (<i>AKA</i> <b>Parameter Bounds</b>) given as a dictionary

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

### Target Function

Here, we define a simple target function called <b><code>simple_func</code></b>
 . The function calculates the sum of inputs <b><code>a</code></b> and <b><code>b</code></b> and returns the result. In this model, <b><code>a</code></b> and <b><code>b</code></b> are the hyperparameters we want to optimize.

In [155]:
def simple_func(a, b):
    return a + b

### Parameter Bounds

The hyperparameter search space is defined as a Python dictionary. Here, we assume that $a \in (1,3)$ and $b \in (4,7)$ . The Bayesian Optimization will explore these range of values.

In [156]:
pbounds = {
      'a': (1, 3)
    , 'b': (4, 7)
}

### Optimizer Function

In [157]:
optimizer = BayesianOptimization(
      simple_func
    , pbounds
    , random_state = 42
)

### Maximization

Let's run an example where we use the <b><code>optimizer</code></b> to find the best values to maximize the target value. The <b><code>.maximize</code></b> method runs the optimization process, repeatedly calling <b><code>simple_func()</code></b> with different values of <b><code>a</code></b> and <b><code>b</code></b> from the search space.

In [158]:
optimizer.maximize(
      n_iter      = 3
    , init_points = 2
)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 8.195     | 1.718     | 6.477     |
| 2         | 7.36      | 2.586     | 4.774     |
| 3         | 6.827     | 1.438     | 5.388     |
| 4         | 6.852     | 1.457     | 5.395     |
| 5         | 8.864     | 2.191     | 6.673     |


### Results

Great, now let's print the best parameters and the associated maximized target.

In [159]:
print(optimizer.max['params'])
print(optimizer.max['target'])

{'a': 2.191138571602373, 'b': 6.673262190236072}
8.864400761838445


## 3.2. LightGBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

### Target Function

Let's build a LightGBM model to test the bayesian optimizer on real data. We will be using the training and test datasets as defined in section 2.

[LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function <b><code>lgb_eval()</code></b> we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as <b><code>num_leaves</code></b> , <b><code>max_depth</code></b> , <b><code>min_child_samples</code></b> , <b><code>min_data_in_leaf</code></b> should be integers.

Apply the Bayesian optimizer to the <b><code>lgb_evl()</code></b> function to identify the best hyperparameters. We will run 10 iterations and set <b><code>init_points=2</code></b> .


In [160]:
cat_cols_idx = [full_df_encoded.columns.get_loc(col_name) for col_name in cat_cols]
cat_cols_idx

[3, 4, 5, 7, 30, 31, 32]

In [161]:
# Define a partial function with fixed arguments (training data and categorical features)
lgb_eval_partial = partial(
      lgb_eval
    , train_data           = X_train
    , train_label          = y_train
    , categorical_features = cat_cols_idx
)

### Parameter Bounds

> <b>NOTE:</b> <b><code>train_data</code></b> , <b><code>train_label</code></b> , and <b><code>cat_cols_idx</code></b> should not be included in the <b><code>pbounds</code></b> dictionary. You will pass them as arguments when calling the <b><code>lgb_eval()</code></b> .

In [162]:
pbounds = {
      'num_leaves'        : (25, 4000)
    , 'max_depth'         : (5, 63)
    , 'lambda_l2'         : (0.0, 0.05)
    , 'lambda_l1'         : (0.0, 0.05)
    , 'min_child_samples' : (50, 10000)
    , 'min_data_in_leaf'  : (100, 2000)
}

### Optimizer Function

To view all available parameters, please refer to the official documentation for <a href="https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.cv.html"><b><code>lightgbm.cv()</code></b></a> .

In [163]:
lgbBO = BayesianOptimization(
      lgb_eval_partial
    , pbounds
    , random_state = 42
)

### Maximization

These are the main parameters of this function:

* <b><code>n_iter</code></b> : This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* <b><code>init_points</code></b> : This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [164]:
lgbBO.maximize(
      n_iter      = 10
    , init_points = 2
)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7154    | 0.009355  | 0.02227   | 10.26     | 2.867e+03 | 647.6     | 3.556e+03 |
| 2         | 0.715     | 0.04064   | 0.002071  | 18.42     | 9.96e+03  | 624.9     | 386.9     |
| 3         | 0.7428    | 0.001761  | 0.03269   | 53.04     | 1.081e+03 | 1.423e+03 | 2.511e+03 |
| 4         | 0.7166    | 0.001326  | 0.007943  | 58.89     | 4.971e+03 | 876.4     | 3.448e+03 |
| 5         | 0.7415    | 0.03343   | 0.01608   | 48.51     | 974.4     | 1.893e+03 | 2.764e+03 |
| 6         | 0.7417    | 0.007527  | 0.02527   | 47.17     | 143.1     | 1.816e+03 | 1.073e+03 |
| 7         | 0.742     | 0.04681   | 0.04302   | 32.34     | 2.311e+03 | 1.787e+03 | 28.84     |
| 8         | 0.7005    | 0.03659   | 0.03754   | 47.41     | 837.3     | 101.7     | 143.2     |
| 9         | 0.7417

The best combination of (hyper)parameters and target value found can be viewed using the <b><code>.max</code></b> method.

In [178]:
print(f"Best result: {lgbBO.max}")

Best result: {'target': 0.7427502968533645, 'params': {'lambda_l1': 0.0017613450752210437, 'lambda_l2': 0.03269029318987155, 'max_depth': 53.041575355823426, 'min_child_samples': 1081.4318097587757, 'min_data_in_leaf': 1423.4418892341832, 'num_leaves': 2511.233702551642}}


### Results

Review the process at each step by using the <b><code>.res</code></b> function.

In [173]:
lgbBO.res

[{'target': 0.7154300040680992,
  'params': {'lambda_l1': 0.009354501638056973,
   'lambda_l2': 0.022274375148079614,
   'max_depth': 10.25580678059873,
   'min_child_samples': 2867.298062509216,
   'min_data_in_leaf': 647.6319100950644,
   'num_leaves': 3556.2483435694476}},
 {'target': 0.714960603472424,
  'params': {'lambda_l1': 0.04064325674958924,
   'lambda_l2': 0.002070789929854655,
   'max_depth': 18.417551423995082,
   'min_child_samples': 9959.903672884064,
   'min_data_in_leaf': 624.9425575946591,
   'num_leaves': 386.9196004708692}},
 {'target': 0.7427502968533645,
  'params': {'lambda_l1': 0.0017613450752210437,
   'lambda_l2': 0.03269029318987155,
   'max_depth': 53.041575355823426,
   'min_child_samples': 1081.4318097587757,
   'min_data_in_leaf': 1423.4418892341832,
   'num_leaves': 2511.233702551642}},
 {'target': 0.7166030504571438,
  'params': {'lambda_l1': 0.0013260420421196684,
   'lambda_l2': 0.007943012203789512,
   'max_depth': 58.88657255799456,
   'min_child_s